In [107]:
from itertools import combinations
from collections import defaultdict, namedtuple, deque
from math import isclose
from operator import eq

import uuid

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Polygon

from PIL import *

rng = np.random.default_rng(seed=1)

grid_count = 5
grid_bounds = [-2, 2]
grid_bounds[1] += 1
#cos_list = [np.cos(x * np.pi / grid_count) for x in np.arange(grid_count)]
#sin_list = [np.sin(x * np.pi / grid_count) for x in np.arange(grid_count)]
cos_list = [np.cos(x * 2 * np.pi / grid_count) for x in np.arange(grid_count)]
sin_list = [np.sin(x * 2 * np.pi / grid_count) for x in np.arange(grid_count)]

#offsets = rng.uniform(size=grid_count)
#offsets = offsets / np.sum(offsets)
offsets = np.repeat(1/grid_count, grid_count)
#offsets = np.repeat(.2, grid_count)
# offsets = np.empty((grid_count,),float)
# offsets[::2] = (1/grid_count)
# offsets[1::2] = -(1/grid_count)
#offsets = np.repeat(0, grid_count)
#offsets = np.repeat(1/grid_count, grid_count)

base_point = np.array([0, 0], dtype=float)

def determine_base_K(p, c, s, o):
    # If it's on a line, lower it to the origin
    if isclose(((c * p[0] + s * p[1]) % 1), o, rel_tol=1e-5, abs_tol=1e-05):
        p_distance = np.sqrt(p[0]**2 + p[1]**2)
        p_normal_origin = [-p_ / p_distance * .5 for p_ in p]
        p = [p + pno for p, pno in zip(p, p_normal_origin)]

    base_k = np.ceil(c * p[0] + s * p[1] - o)
    return base_k

base_k_values = []
for c, s, o in zip(cos_list, sin_list, offsets):
    base_k_values.append(determine_base_K(base_point, c, s, o))

mid_point = np.array([0, 0], dtype=float)
for k, c, s in zip(base_k_values, cos_list, sin_list):
    mid_point += k * np.array([c, s])

def determine_K(c, x, s, y, o):
    return int(np.ceil(c * x + s * y - o))

class Point:
    def __init__(self, k_values):
        self.k_values = k_values
        
        self.location = np.array([0, 0], dtype=float)
        for k, c, s in zip(k_values, cos_list, sin_list):
            self.location += k * np.array([c, s])
        self.location -= mid_point

points = {}

edges = defaultdict(list)

class Line:
    # a*x + b*y = c
    def __init__(self, a, b, o, k, grid):
        self.a = a
        self.b = b
        self.c = o + k
        self.offset = o
        self.k = int(k)
        self.grid = grid

        self.first_angle = np.arctan2(-a, b)
        if self.first_angle < 0:
            self.second_angle = self.first_angle + np.pi
        else:
            self.second_angle = self.first_angle - np.pi

        # crossing counter clockwise, first angle is k + 1, second is k
        # because of how np.arctan2 works?
        Angle = namedtuple('Angle', ['value', 'k', 'grid'])

        self.angles = [Angle(self.first_angle, self.k + 1, grid), 
                       Angle(self.second_angle, self.k, grid)]

        # Find closes point to the origin
        self.origin_x = (self.a * self.c) / (self.a**2 + self.b**2)
        self.origin_y = (self.b * self.c) / (self.a**2 + self.b**2)

        self.perpendicular_angle = np.arctan2(b, a)

        # Keep this stuff in here for testing, but add parameter to exclude it later
        self.p1 = [np.cos(self.perpendicular_angle) * .25 + self.origin_x,
              np.sin(self.perpendicular_angle) * .25 + self.origin_y ]
        self.p2 = [np.cos(self.perpendicular_angle) * -.25 + self.origin_x,
              np.sin(self.perpendicular_angle) * -.25 + self.origin_y ]

        self.K = [self.determine_K(self.p1), self.determine_K(self.p2)]

    def __str__(self):
        return f'a={self.a}, b={self.b}, offset={self.offset}, k={self.k}, grid={self.grid}'

    def determine_K(self, p):
        return np.ceil(self.a * p[0] + self.b * p[1] - self.offset)

    # Build function to get lines that hit boundaries
    # Maybe use build intersection function and use a line for each boundary!!!???
    def determine_intersection(self, other_line):

        denom = (self.a * other_line.b) - (other_line.a * self.b)
        if isclose(denom, 0, rel_tol=1e-5, abs_tol=1e-05):
            # Determine if it's the same line

            if isclose(other_line.a, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.a, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                a_proportion = 0
            else:
                a_proportion = self.a / other_line.a
            
            if isclose(other_line.b, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.b, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                b_proportion = 0
            else:
                b_proportion = self.b / other_line.b

            if a_proportion and b_proportion:
                if not isclose(a_proportion, b_proportion):
                    return None

            if isclose(other_line.c, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.c, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                c_proportion = 0
            else:
                c_proportion = self.c / other_line.c

            if a_proportion and c_proportion:
                if not isclose(a_proportion, c_proportion, rel_tol=1e-5, abs_tol=1e-05):
                    return None
            
            if b_proportion and c_proportion:
                if not isclose(b_proportion, c_proportion, rel_tol=1e-5, abs_tol=1e-05):
                    return None

            raise OverlappingLines(self, other_line)
        else:
            x = (other_line.b * self.c) - (self.b * other_line.c)
            x = x / denom

            y = (self.a * other_line.c) - (other_line.a * self.c)
            y = y / denom

            return (x, y)
        
class OverlappingLines(Exception):
    def __init__(self, line_1, line_2, msg="These two lines are the same line."):
        self.line_1 = line_1
        self.line_2 = line_2
        self.msg = msg
        super().__init__(self.msg)

    def __str__(self):
        return f'{self.msg}\n\t{self.line_1}\n\t{self.line_2}'
    
class Tile:
    def __init__(self, tile_points):
        self.tile_points = tile_points
        self.tile_group = 0

        self.tile_id = uuid.uuid4()
        self.connected = False

        # Don't need to compare distances between angles, always 1 by construction
        self.angles = []
        for i, tile_point in enumerate(self.tile_points):
            previous_point = self.tile_points[(i - 1) % len(self.tile_points)]
            next_point = self.tile_points[(i + 1) % len(self.tile_points)]

            point_location = points[tile_point].location
            previous_point_location = points[previous_point].location
            next_point_location = points[next_point].location

            in_vector = point_location - previous_point_location
            out_vector = next_point_location - point_location
            angle = np.arctan2(in_vector[0]*out_vector[1]-in_vector[1]*out_vector[0],
                            in_vector[0]*out_vector[0]+in_vector[1]*out_vector[1])

            self.angles.append(angle)

    def compare_angles(self, other_polygon):
        if len(self.angles) != len(other_polygon.angles):
            return False

        start = 0
        matched = False
        while start < len(self.angles) and not matched:
            for i in range(len(self.angles)):
                place = (i + start) % len(self.angles)
                if not isclose(self.angles[place], other_polygon.angles[i], rel_tol=1e-5, abs_tol=1e-05):
                    break
            if i == len(self.angles)-1:
                matched = True
            start += 1
        
        return matched
                
grids = []
for i, (s, c, o, bk) in enumerate(zip(sin_list, cos_list, offsets, base_k_values)):
    for k in np.arange(*grid_bounds):
        grids.append(Line(c, s, o, bk + k, i))

intersections = defaultdict(set)
for pair in combinations(grids, 2):
    l1 = pair[0]
    l2 = pair[1]
    
    if l1.grid == l2.grid:
        continue

    try:
        intersection = l1.determine_intersection(l2)
        if intersection:
            intersections[intersection].update([l1, l2])
    except OverlappingLines as e: 
        print(e)

base_tiles = []
tiles = {}

for intersection_points, intersection_lines in intersections.items():

    # Maybe, get all the angles in order
    angles = list(intersection_lines)[0].angles.copy()
    angles.extend(list(intersection_lines)[1].angles.copy())
    angles.sort()

    # #start with right above (or below) the first one, (maybe need to get to second+ line to get all the K's) then continue looping through
    # Set up K values for the other lines
    ks = [determine_K(c, intersection_points[0], s, intersection_points[1], o) for c, s, o in zip(cos_list, sin_list, offsets)]

    # Now spin around to ensure the ks are correct on the lines that intersect
    for angle in angles:
        ks[angle.grid] = angle.k

    tile_points = []
    for angle in angles:
        ks[angle.grid] = angle.k

        k_values = tuple(ks)
        if not k_values in points:
            points[k_values] = Point(k_values)

        tile_points.append(k_values)

    t = Tile(tile_points)

    for t_current, t_next in zip(t.tile_points[:-1], t.tile_points[1:]):
        # Check left to right, then down to up
        if points[t_current].location[1] < points[t_next].location[1]:
            edges[(t_current, t_next)].append(t.tile_id)
        elif points[t_next].location[1] < points[t_current].location[1]:
            edges[(t_next, t_current)].append(t.tile_id)
        elif points[t_current].location[0] < points[t_next].location[0]:
            edges[(t_current, t_next)].append(t.tile_id)
        else:
            edges[(t_next, t_current)].append(t.tile_id)
    t_current = t.tile_points[-1]
    t_next = t.tile_points[0]
    if points[t_current].location[1] < points[t_next].location[1]:
        edges[(t_current, t_next)].append(t.tile_id)
    elif points[t_next].location[1] < points[t_current].location[1]:
        edges[(t_next, t_current)].append(t.tile_id)
    elif points[t_current].location[0] < points[t_next].location[0]:
        edges[(t_current, t_next)].append(t.tile_id)
    else:
        edges[(t_next, t_current)].append(t.tile_id)

    bt_match = False
    for bt in base_tiles:
        if bt.compare_angles(t):
            t.tile_group = bt.tile_group
            bt_match = True
            break

    if not bt_match:
        t.tile_group = len(base_tiles)
        base_tiles.append(t)

    tiles[t.tile_id] = t

colors = ["blue", "green", "yellow", "orange", "red"]



In [108]:
# I really need to make the tiles into a graph instead
# Use points to make edges when creating the tiles, keep track of when the edges are hit again, mark those tiles as neighbors in the graph

neighbors = defaultdict(list)
for tile_list in edges.values():
    if len(tile_list) == 2:
        neighbors[tile_list[0]].append(tile_list[1])
        neighbors[tile_list[1]].append(tile_list[0])
    elif len(tile_list) > 2:
        print('Error')

# Find center-ish tile
center_tile = tiles[list(tiles.keys())[0]]
center_point = center_tile.tile_points[0]
center_distance = points[center_point].location[0]**2 + points[center_point].location[1]**2
for tile in tiles.values():
    tile_point = tile.tile_points[0]
    tile_distance = points[tile_point].location[0]**2 + points[tile_point].location[1]**2
    if tile_distance < center_distance:
        center_tile = tile
        center_point = tile_point
        center_distance = tile_distance


In [ ]:
center_tile.connected = True
connected_tiles = deque([center_tile])

while connected_tiles:
    current_tile = connected_tiles.pop()
    for neighbor in neighbors[current_tile.tile_id]:
        neighbor_tile = tiles[neighbor]
        if not neighbor_tile.connected:
            neighbor_tile.connected = True
            connected_tiles.append(neighbor_tile)

# Now, delete the not connected tiles
# Also delete edges on those tiles
# Find closest point on edges without two tiles
# Use this distance to origin as diagnol for image width

In [113]:
last_no_white = 10

fig, ax = plt.subplots(figsize=(5, 5))

for t in tiles.values():
    locations = []
    for p in t.tile_points:
        locations.append(points[p].location)

    # neighbor_count = len(neighbors[t.tile_id])
        
    p = Polygon(locations, facecolor = colors[t.connected], edgecolor="k")
    ax.add_patch(p)
    
# for p in points.values():
#     ax.text(p.location[0], p.location[1], p.k_values, size=5)

ax.set_aspect('equal')
plt.axis([-last_no_white, last_no_white, -last_no_white, last_no_white])
plt.tight_layout(pad=0)

graph_file = "output/graph.png"
plt.savefig(graph_file, dpi=150)
plt.close()

In [ ]:
# Try making the background transparent and pulling that instead of white
# remove tiles not touching other ones
# find border, set closest border point to be the diagonal length


-33.27050983124842


In [ ]:
points = {}

def compare_points(a, b):
    if (isclose(a[0], b[0], rel_tol=1e-5, abs_tol=1e-05) &
        isclose(a[1], b[1], rel_tol=1e-5, abs_tol=1e-05)):
        return True
    else:
        return False
    
Dont need to do any of this because they would have the same K values
Dictionary that maps Ks to a point class, create point class
Use edges to connect tiles from center tile delete those that aren't connected, make a graph for this or something'
Center tile will have closest point to 0, 0, select one if ties
Might not need that, just find closest edge to center and use that distance as the square diagnol
    


0

In [14]:
my_list = [10, 20, 30, 40, 50]
for current_value, next_value in zip(my_list[:-1], my_list[1:]):
    print(f"Current: {current_value}, Next: {next_value}")
if my_list:
    print(f"Current: {my_list[-1]}, Next: None") #handling last element

Current: 10, Next: 20
Current: 20, Next: 30
Current: 30, Next: 40
Current: 40, Next: 50
Current: 50, Next: None
